###Bem-vindo(a) a aula de Agrupamento e Slicing com Pandas

Tópicos da aula:
- Agrupamento e Slicing
- Indexação no Pandas
- Manipulação do Index
- Seleção condicional
- Agrupamentos
- Multi-índices

Vamos carregar nosso *DataSet*


In [110]:
import pandas as pd

In [111]:
pd.options.display.float_format = '{:,.2f}'.format

In [112]:
dataset = r'https://raw.githubusercontent.com/FelipeHardmann/Curso_DataScience/main/dataset/produtos.csv'

In [113]:
df = pd.read_csv(dataset)

In [114]:
df.sample(5)

,Produto,Cor,Valor,Quantidade,Sexo,Idade,Data da Venda,Canal
0,Camiseta,Preta,29.99,50,Masculino,30,2023-08-01,Loja Física
15,Calça Jeans,Preta,59.99,37,Feminino,28,2023-08-16,Loja Online
7,Vestido,Azul,39.99,38,Feminino,29,2023-08-08,Loja Online
4,Camiseta,Verde,29.99,45,Masculino,32,2023-08-05,Loja Física
11,Blusa,Amarelo,34.99,48,Feminino,31,2023-08-12,Loja Online


In [115]:
df.dtypes

Produto           object
Cor               object
Valor            float64
Quantidade         int64
Sexo              object
Idade              int64
Data da Venda     object
Canal             object
dtype: object

Agora que vimos todas as colunas que temos no nosso *DataSet*, precisamos alterar a tipagem para otimizarmos nossas consultas e trabalharmos da maneira correta.

In [116]:
df['Valor'] = pd.to_numeric(df['Valor'], downcast='float')

In [117]:
df['Valor'].dtypes

dtype('float32')

In [118]:
df['Quantidade'] = pd.to_numeric(df['Quantidade'], downcast='integer')

In [119]:
df['Quantidade'].dtypes

dtype('int8')

In [120]:
df['Idade'] = pd.to_numeric(df['Idade'], downcast='integer')

In [121]:
df['Idade'].dtypes

dtype('int8')

##Indexação no Pandas

####Seleção baseado em índice

A indexação do Pandas funciona com dois paradigmas. A primeira é a seleção baseada em índice: selecionar dados com base em sua posição numérica nos dados. O iloc segue este paradigma.

A seleção por **iloc** é simples, ele seleciona por números inteiros das linhas, arrays ou por slice.

```
df.iloc[<linhas>, <colunas>]
```

Ou seja, os dois argumentos são: linhas e colunas.


Então, para selecionarmos a primeira linha de um *DataFrame*, basta utilizarmos o **iloc** passando o index posicional.

In [122]:
df.iloc[0]

Produto             Camiseta
Cor                    Preta
Valor                  29.99
Quantidade                50
Sexo               Masculino
Idade                     30
Data da Venda     2023-08-01
Canal            Loja Física
Name: 0, dtype: object

Dessa maneira, vemos uma *Series* acima.

In [123]:
df.iloc[0].to_frame().transpose()

,Produto,Cor,Valor,Quantidade,Sexo,Idade,Data da Venda,Canal
0,Camiseta,Preta,29.99,50,Masculino,30,2023-08-01,Loja Física


###Obs:

Tanto o **loc** quanto o **iloc** trabalham primeiro com linha e depois com a coluna.

Isto significa que é um pouco mais fácil recuperar linhas e um pouco mais difícil recuperar colunas. Para obter uma coluna com iloc, devemos escrever assim: ```df.iloc[:, 0]```, onde o seletor : significa tudo.

In [124]:
df.iloc[:, 0]

0         Camiseta
1      Calça Jeans
2            Tênis
3          Vestido
4         Camiseta
5           Shorts
6            Tênis
7          Vestido
8             Saia
9          Jaqueta
10           Calça
11           Blusa
12          Camisa
13    Shorts Jeans
14           Tênis
15     Calça Jeans
16         Vestido
17        Camiseta
18      Saia Jeans
Name: Produto, dtype: object

Porém, o **iloc** é muito forte quando combinamos ele, por exemplo:

In [125]:
df.iloc[0:5, 1]

0       Preta
1        Azul
2      Branco
3    Vermelho
4       Verde
Name: Cor, dtype: object

Perceba que acima estamos pegando os dados da linha 0 a 4, da coluna posição com indíce 1.

Além disso, podemos passar um intervalo de valores através de uma lista

In [126]:
df.iloc[[0, 1, 2, 3], 0]

0       Camiseta
1    Calça Jeans
2          Tênis
3        Vestido
Name: Produto, dtype: object

Ou seja, pegamos as linhas 0, 1, 2 e 3, da primeira coluna do *DataFrame*, que nesse caso é produto.

In [127]:
df.iloc[[*range(0, 10)], 0]

0       Camiseta
1    Calça Jeans
2          Tênis
3        Vestido
4       Camiseta
5         Shorts
6          Tênis
7        Vestido
8           Saia
9        Jaqueta
Name: Produto, dtype: object

Podemos fazer essas consultas até mesmo com o nosso *list comprehension*

Dessa maneira, como é igual ao trabalho de lista, podemos pegar também os últimos dados de uma forma mais fácil.


In [128]:
df.iloc[-5:]

,Produto,Cor,Valor,Quantidade,Sexo,Idade,Data da Venda,Canal
14,Tênis,Vermelho,69.99,29,Masculino,29,2023-08-15,Loja Física
15,Calça Jeans,Preta,59.99,37,Feminino,28,2023-08-16,Loja Online
16,Vestido,Amarelo,39.99,33,Feminino,30,2023-08-17,Loja Física
17,Camiseta,Cinza,29.99,49,Masculino,27,2023-08-18,Loja Online
18,Saia Jeans,Azul,24.99,44,Feminino,24,2023-08-19,Loja Física


Perceba que ao chamarmos dados sem especificarmos a coluna, ele vem da forma bidimensional, ou seja, um *DataFrame*.

### Seleção baseada em rótulo

O segundo paradigma para seleção de atributos é o seguido pelo operador **loc**: seleção baseada em rótulos.

Nesse paradigma, é o valor do índice de dados, não sua posição, que importa.

Dessa forma, podemos entender que o **loc** é primariamente baseado em *labels* das colunas.

A sintáxe básica do **loc** é da seguinte forma:

```
df.loc[<posição>, 'coluna']
```

Vamos ver alguns exemplos.


Podemos pegar o primeiro dado da coluna **Cor**

In [129]:
df.loc[0, 'Cor']

'Preta'

POrém, temos uma forma mis interessante de usar o **loc**, que é com *array*(lista).

Podemos usar da seguinte forma:

In [130]:
df.loc[:, ['Cor', 'Idade', 'Produto']]

,Cor,Idade,Produto
0,Preta,30,Camiseta
1,Azul,25,Calça Jeans
2,Branco,28,Tênis
3,Vermelho,22,Vestido
4,Verde,32,Camiseta
5,Amarelo,24,Shorts
6,Preto,26,Tênis
7,Azul,29,Vestido
8,Rosa,27,Saia
9,Marrom,35,Jaqueta


Ou seja, essa usabilidade do **loc** é muito interessante pois podemos criar novos *DataFrames*.

In [131]:
new_df = df.loc[:, ['Cor', 'Produto', 'Valor']]

In [132]:
new_df

,Cor,Produto,Valor
0,Preta,Camiseta,29.99
1,Azul,Calça Jeans,59.99
2,Branco,Tênis,79.99
3,Vermelho,Vestido,39.99
4,Verde,Camiseta,29.99
5,Amarelo,Shorts,19.99
6,Preto,Tênis,79.99
7,Azul,Vestido,39.99
8,Rosa,Saia,24.99
9,Marrom,Jaqueta,69.99


Dessa maneira, podemos ir criando variáveis para guardar determinado *DataFrame*.

E essa seria a melhor forma para trabalharmos, já que, é mais interessante fazermos nossas análises de forma separada, sem alterar tanto o *DataFrame* principal.

Vamos ver mais alguns exemplos:

In [133]:
df.loc[0:5, 'Sexo']

0    Masculino
1     Feminino
2    Masculino
3     Feminino
4    Masculino
5     Feminino
Name: Sexo, dtype: object

### Conclusão sobre o **loc** e **iloc**

Os dois recursos são muito úteis para a nossa análise exploratória, o **loc** é um pouco mais versátil, pelo fato de selecionarmos a coluna pelo seu nome, diferente do **iloc** que depende dos números dos seus índices.

Outra questão é que os dois comandos usam esquema de indexação diferente, onde i **iloc** utiliza o esquema de indexação *stdlib* do *Python*, onde o primeiro número é incluído e o último não, diferente do **loc** que faz a indexação inclusive do último número.

Vamos ver um exemplo sobre isso.

In [134]:
print(f'iloc: {len(df.iloc[0:10])} -----> loc: {len(df.loc[0:10])}')

iloc: 10 -----> loc: 11


##Manipulando o index

A seleção baseada em rótulos possui sua força através dos rótulos do índice.

Nesse caso, temos o método *set_index()*, que pode ser usado para criarmos um índice da maneira que quisermos.

Vamos ver alguns exemplos:

In [135]:
df.set_index('Data da Venda')[:10]

,Produto,Cor,Valor,Quantidade,Sexo,Idade,Canal
Data da Venda,,,,,,,
2023-08-01,Camiseta,Preta,29.99,50,Masculino,30,Loja Física
2023-08-02,Calça Jeans,Azul,59.99,40,Feminino,25,Loja Online
2023-08-03,Tênis,Branco,79.99,30,Masculino,28,Loja Física
2023-08-04,Vestido,Vermelho,39.99,35,Feminino,22,Loja Online
2023-08-05,Camiseta,Verde,29.99,45,Masculino,32,Loja Física
2023-08-06,Shorts,Amarelo,19.99,60,Feminino,24,Loja Online
2023-08-07,Tênis,Preto,79.99,32,Masculino,26,Loja Física
2023-08-08,Vestido,Azul,39.99,38,Feminino,29,Loja Online
2023-08-09,Saia,Rosa,24.99,55,Feminino,27,Loja Física


Dessa forma, estamos gerando um *DataFrame* onde a indexação dele é a Data da Venda.

Lembrando que se quisermos salvar na memória, precisamos passar o parâmetro *inplace=True*.

###Seleção Condicional

O método **loc** funciona muito bem através de seleções condicionais, com ele deixamos nossa análise muito mais robusta.

Por exemplo, podemos mostrar apenas os pessoas com mais de 30 anos, roupas mais barata pegando apenas aquelas que são menos de 25 e assim por diante.

No caso, como já vimos, quando utilizamos uma estrutura condicional de forma pura, ela irá retornar apenas **True** ou **False**.

In [136]:
df['Idade'] > 25

0      True
1     False
2      True
3     False
4      True
5     False
6      True
7      True
8      True
9      True
10    False
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18    False
Name: Idade, dtype: bool

Porém, podemos salvar essa *Serie* dentro de uma variável e utilizar ela.

In [137]:
maior_25 = df['Idade'] > 25

In [138]:
df.loc[maior_25]

,Produto,Cor,Valor,Quantidade,Sexo,Idade,Data da Venda,Canal
0,Camiseta,Preta,29.99,50,Masculino,30,2023-08-01,Loja Física
2,Tênis,Branco,79.99,30,Masculino,28,2023-08-03,Loja Física
4,Camiseta,Verde,29.99,45,Masculino,32,2023-08-05,Loja Física
6,Tênis,Preto,79.99,32,Masculino,26,2023-08-07,Loja Física
7,Vestido,Azul,39.99,38,Feminino,29,2023-08-08,Loja Online
8,Saia,Rosa,24.99,55,Feminino,27,2023-08-09,Loja Física
9,Jaqueta,Marrom,69.99,28,Masculino,35,2023-08-10,Loja Online
11,Blusa,Amarelo,34.99,48,Feminino,31,2023-08-12,Loja Online
12,Camisa,Verde,44.99,36,Masculino,33,2023-08-13,Loja Física
13,Shorts Jeans,Azul,19.99,58,Feminino,26,2023-08-14,Loja Online


Perceba que ele ignora os dados que viriam como *False* e gera um novo *DataFrame* com os dados que estavam como *True*.

Seguindo essa linha, podemos diminuir ainda mais esse *DataFrame*, podemos pegar agora somente as pessoas que são do sexo Masculino.

Nesse caso, podemos usar os operadores lógicos do Python.

**and** -> &

**or** -> |

```
df.loc[(condicao 1) < & ou | > (condicao 2)]
```

In [139]:
maior_25 = df['Idade'] > 25
masculino = df['Sexo'] == 'Masculino'

In [140]:
df.loc[(maior_25) & (masculino)]

,Produto,Cor,Valor,Quantidade,Sexo,Idade,Data da Venda,Canal
0,Camiseta,Preta,29.99,50,Masculino,30,2023-08-01,Loja Física
2,Tênis,Branco,79.99,30,Masculino,28,2023-08-03,Loja Física
4,Camiseta,Verde,29.99,45,Masculino,32,2023-08-05,Loja Física
6,Tênis,Preto,79.99,32,Masculino,26,2023-08-07,Loja Física
9,Jaqueta,Marrom,69.99,28,Masculino,35,2023-08-10,Loja Online
12,Camisa,Verde,44.99,36,Masculino,33,2023-08-13,Loja Física
14,Tênis,Vermelho,69.99,29,Masculino,29,2023-08-15,Loja Física
17,Camiseta,Cinza,29.99,49,Masculino,27,2023-08-18,Loja Online


Perceba que diminuímos ainda mais a nossa análise.

###Obs:

O *Pandas* vem com alguns seletores condicionais embutidos.

**isin** e **isnull(notnull)**

O **isin** permite selecionarmos valores que estão presentes de acordo com uma lista de valores.

In [141]:
df.loc[df['Idade'].isin([20, 25, 30])]

,Produto,Cor,Valor,Quantidade,Sexo,Idade,Data da Venda,Canal
0,Camiseta,Preta,29.99,50,Masculino,30,2023-08-01,Loja Física
1,Calça Jeans,Azul,59.99,40,Feminino,25,2023-08-02,Loja Online
16,Vestido,Amarelo,39.99,33,Feminino,30,2023-08-17,Loja Física


Dessa maneira, ele me retorna todos os dados do *DataFrame* que possuem a Idade igual a 20 ou 25 ou 30.

Já o **isnull**, vai retornar os dados null, se tivermos.

In [142]:
df.loc[df['Idade'].isnull()]

,Produto,Cor,Valor,Quantidade,Sexo,Idade,Data da Venda,Canal


Como possuímos Idades em toda nossa coluna Idade, ele não vai retornar nada.

### Agrupamentos

Muitas vezes queremos agrupar os dados e, em seguinda, fazer algo específico para o grupo em que os dados estão.

Assim, temos os método **_groupby()_**. Essa função vai agrupar os dados e permitir a execução de operações para cada grupo criado, com base na coluna e/ou condição desejada em grupos e aplica a função desejada para o grupo, com o intuito de ter uma única saída.


Um método que é muito interessante usar com o **_groupby()_** é **agg()**, que é o método que vimos na aula passada, que permite executarmos várias funções, de forma simultânea, em um *DataFrame*.

In [143]:
df.groupby('Produto')['Valor'].agg(['min', 'max', 'sum'])

,min,max,sum
Produto,,,
Blusa,34.99,34.99,34.99
Calça,49.99,49.99,49.99
Calça Jeans,59.99,59.99,119.98
Camisa,44.99,44.99,44.99
Camiseta,29.99,29.99,89.97
Jaqueta,69.99,69.99,69.99
Saia,24.99,24.99,24.99
Saia Jeans,24.99,24.99,24.99
Shorts,19.99,19.99,19.99


Vamos entender o que aconteceu acima.

Primeiro, pegamos os valores por Produto, depois chamamos a função **agg()**, para agruparmos as funções que queremos, depois chamamos os menores valores de cada produto, os maiores e a soma total de valores.

###Vamos utilizar outro *DataSet*.

In [144]:
dataset = r'https://raw.githubusercontent.com/jennybc/gapminder/master/data-raw/08_gap-every-five-years.tsv'
df_paises = pd.read_csv(dataset, sep='\t')

In [145]:
df_paises

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.80,8425333,779.45
1,Afghanistan,Asia,1957,30.33,9240934,820.85
2,Afghanistan,Asia,1962,32.00,10267083,853.10
3,Afghanistan,Asia,1967,34.02,11537966,836.20
4,Afghanistan,Asia,1972,36.09,13079460,739.98
...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.35,9216418,706.16
1700,Zimbabwe,Africa,1992,60.38,10704340,693.42
1701,Zimbabwe,Africa,1997,46.81,11404948,792.45
1702,Zimbabwe,Africa,2002,39.99,11926563,672.04


Agora, vamos mudar o nome de todas as colunas para o português.

In [146]:
df_paises.rename(columns={'country':'pais',
                           'continent':'continente',
                           'year':'ano',
                           'lifeExp':'expVida',
                           'pop':'populacao',
                           'gdpPercap':'pibPercap',
                          }, inplace=True)

In [147]:
df_paises.head()

,pais,continente,ano,expVida,populacao,pibPercap
0,Afghanistan,Asia,1952,28.80,8425333,779.45
1,Afghanistan,Asia,1957,30.33,9240934,820.85
2,Afghanistan,Asia,1962,32.00,10267083,853.10
3,Afghanistan,Asia,1967,34.02,11537966,836.20
4,Afghanistan,Asia,1972,36.09,13079460,739.98


In [148]:
df_paises.groupby('pais')['expVida'].agg(['mean', 'max', 'min'])

,mean,max,min
pais,,,
Afghanistan,37.48,43.83,28.80
Albania,68.43,76.42,55.23
Algeria,59.03,72.30,43.08
Angola,37.88,42.73,30.02
Argentina,69.06,75.32,62.48
...,...,...,...
Vietnam,57.48,74.25,40.41
West Bank and Gaza,60.33,73.42,43.16
"Yemen, Rep.",46.78,62.70,32.55


Agora, temos a média dos últimos anos da expectativa de vida de todos os países do no nosso *DataFrame*.

E podemos fazer isso em diversas consultas, portanto, a função **_groupby_** é muito forte e muita usada juntamente com a função **agg()**

Vamos agora olhar a expectativa do Brasil.

In [149]:
df_exp_vida = df_paises.groupby('pais')['expVida'].agg(['mean', 'max', 'min'])

In [150]:
df_exp_vida.loc['Brazil'].to_frame().transpose()

,mean,max,min
Brazil,62.24,72.39,50.92


###Multi-índices

Agora, vamos trabalhar com Multi-índices, em todos os exemplos trabalhamos, apenas, com um índice de rótulo único de um *DataFrame* ou *Series*.

E quando estamos trabalhando com **groupby()**, dependendo da operação que executamos, às vezes, resultará em um multi-índice.

Usando o *DataFrame* acima, poderíamos dividir os continentes e chamar o país por continente.

In [151]:
df_paises.groupby(['continente', 'pais'])['expVida'].agg(['max', 'min', 'mean'])

max   min  mean
continente pais                            
Africa     Algeria        72.30 43.08 59.03
           Angola         42.73 30.02 37.88
           Benin          56.73 38.22 48.78
           Botswana       63.62 46.63 54.60
           Burkina Faso   52.30 31.98 44.69
...                         ...   ...   ...
Europe     Switzerland    81.70 69.62 75.57
           Turkey         71.78 43.59 59.70
           United Kingdom 79.42 69.18 73.92
Oceania    Australia      81.23 69.12 74.66
           New Zealand    80.20 69.39 73.99

[142 rows x 3 columns]

Agora, que colocamos os países em cada continente, vamos buscar todos os países da América.

In [152]:
df_continente = df_paises.groupby(['continente', 'pais'])['expVida'].agg(['max', 'min', 'mean'])

In [153]:
df_america = df_continente.loc['Americas']

In [154]:
df_america

,max,min,mean
pais,,,
Argentina,75.32,62.48,69.06
Bolivia,65.55,40.41,52.50
Brazil,72.39,50.92,62.24
Canada,80.65,68.75,74.90
Chile,78.55,54.74,67.43
Colombia,72.89,50.64,63.90
Costa Rica,78.78,57.21,70.18
Cuba,78.27,59.42,71.05
Dominican Republic,72.23,45.93,61.55


Vamos fazer mais exemplos, agora vamos voltar a usar nosso *DataFrame* de produtos.

In [155]:
df.head()

,Produto,Cor,Valor,Quantidade,Sexo,Idade,Data da Venda,Canal
0,Camiseta,Preta,29.99,50,Masculino,30,2023-08-01,Loja Física
1,Calça Jeans,Azul,59.99,40,Feminino,25,2023-08-02,Loja Online
2,Tênis,Branco,79.99,30,Masculino,28,2023-08-03,Loja Física
3,Vestido,Vermelho,39.99,35,Feminino,22,2023-08-04,Loja Online
4,Camiseta,Verde,29.99,45,Masculino,32,2023-08-05,Loja Física


In [156]:
df.groupby(['Produto', 'Cor'])['Valor'].agg(['sum', 'mean'])

sum  mean
Produto      Cor                 
Blusa        Amarelo  34.99 34.99
Calça        Salmão   49.99 49.99
Calça Jeans  Azul     59.99 59.99
             Preta    59.99 59.99
Camisa       Verde    44.99 44.99
Camiseta     Cinza    29.99 29.99
             Preta    29.99 29.99
             Verde    29.99 29.99
Jaqueta      Marrom   69.99 69.99
Saia         Rosa     24.99 24.99
Saia Jeans   Azul     24.99 24.99
Shorts       Amarelo  19.99 19.99
Shorts Jeans Azul     19.99 19.99
Tênis        Branco   79.99 79.99
             Preto    79.99 79.99
             Vermelho 69.99 69.99
Vestido      Amarelo  39.99 39.99
             Azul     39.99 39.99
             Vermelho 39.99 39.99

Dessa maneira, conseguimos ver todos os produtos, por possuirmos um *DataFrame* pequeno, com poucos dados para analisar.

### Vamos entender mai sobre as funções *set_index()* e *reset_index()*.

In [157]:
df2 = pd.DataFrame(
    {
        'Turmas': [315, 319, 419, 713, 718],
        'Módulos': ['Python', 'Python', 'Python', 'Python', 'Python'],
        'Horário de Início': ['15:30', '19:00', '19:00', '13:30', '18:30']
    }
)

df2

,Turmas,Módulos,Horário de Início
0,315,Python,15:30
1,319,Python,19:00
2,419,Python,19:00
3,713,Python,13:30
4,718,Python,18:30


In [158]:
df2.set_index?

In [159]:
df2.set_index('Turmas')

,Módulos,Horário de Início
Turmas,,
315,Python,15:30
319,Python,19:00
419,Python,19:00
713,Python,13:30
718,Python,18:30


Devemos lembrar de passar o nosso *inplace*, para salvarmos na memória do computador.

In [160]:
df2.set_index('Turmas', inplace=True)

In [161]:
df2

,Módulos,Horário de Início
Turmas,,
315,Python,15:30
319,Python,19:00
419,Python,19:00
713,Python,13:30
718,Python,18:30


In [162]:
df2.iloc[0]

Módulos              Python
Horário de Início     15:30
Name: 315, dtype: object

Agora nosso index é a turma em questão

Assim, como temos uma função para definir o index, temos uma função para resetar o index.

In [163]:
df2.reset_index(inplace=True)

In [164]:
df2

,Turmas,Módulos,Horário de Início
0,315,Python,15:30
1,319,Python,19:00
2,419,Python,19:00
3,713,Python,13:30
4,718,Python,18:30


##Agora, vamos praticar

In [166]:
dataset2 = r'https://raw.githubusercontent.com/rafaelpuyau/infinity_school/main/ds/datasets/atividade_casa.csv'
df_atividade = pd.read_csv(dataset2)

In [169]:
df_atividade.sample(10)

,Produto,Cor,Valor,Quantidade,Sexo,Idade,Data da Venda,Canal
85420,Camisa,Roxa,168.92,4,Feminino,19,2022-05-30,Loja Virtual
52904,Short,Vermelha,152.71,2,Feminino,47,2022-06-16,Loja Física
89976,Saia,Azul,143.96,1,Feminino,49,2022-07-11,Loja Virtual
25719,Calça,Amarela,52.78,3,Masculino,32,2022-01-02,Loja Física
70089,Saia,Azul,129.07,3,Feminino,40,2022-03-29,Loja Virtual
89021,Saia,Azul,159.22,1,Feminino,19,2022-02-10,Loja Virtual
60956,Calça,Verde,154.26,3,Masculino,38,2022-01-30,Loja Virtual
35253,Camisa,Verde,102.74,3,Masculino,27,2022-06-29,Loja Física
12066,Short,Azul,171.49,1,Masculino,46,2022-05-23,Loja Virtual
7278,Camiseta,Amarela,80.75,2,Masculino,40,2022-04-21,Loja Virtual


Questão 1:

Quantos produtos tem o valor maior que R$100,00?

Questão 2:

Quantas saias abaixo de R$70,00 foram compradas?